<a href="https://colab.research.google.com/github/mucarii/Transfer-Learning/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importar bibliotecas

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

## Carregar e Pré-processar o Dataset

In [13]:
import tensorflow_datasets as tfds

# Carregar o dataset Cats vs Dogs
(ds_train, ds_val), ds_info = tfds.load('cats_vs_dogs',
                                        split=['train[:80%]', 'train[80%:]'],
                                        as_supervised=True,
                                        with_info=True)

# Função de pré-processamento
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224)) # Redimensionar para tamanho compatível com a rede
    image = image / 255.0  # Normalizar
    return image, label

# Aplicar o pré-processamento
ds_train = ds_train.map(preprocess).batch(32).prefetch(1)
ds_val = ds_val.map(preprocess).batch(32).prefetch(1)

## Carregar o Modelo Pré-treinado

In [14]:
# Carregar o modelo pré-treinado (sem a última camada)
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

# Congelar as camadas do modelo base
base_model.trainable = False

## Construir a Rede Neural com Transfer Learning

In [18]:
# Construir o modelo final
model = models.Sequential([
    base_model,  # Modelo pré-treinado
    layers.GlobalAveragePooling2D(),  # Camada de pooling global
    layers.Dense(128, activation='relu'),  # Camada densa com 128 neurônios e ativação ReLU
    layers.Dense(1, activation='sigmoid')  # Camada de saída para duas classes (gatos e cachorros) com ativação sigmoide
])

# Antes de compilar, definir explicitamente o formato de entrada, incluindo o tamanho do batch:
input_shape = (None, 224, 224, 3)  # Definir o formato de entrada compatível com o MobileNetV2, incluindo o batch size
model.build(input_shape=input_shape)  # Construir o modelo com o formato de entrada correto

# Compilar o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])  # Usar o otimizador Adam, a função de perda binária e métrica de acurácia

# Visualizar a estrutura do modelo
model.summary()  # Exibir o resumo da arquitetura do modelo

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_5           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

## Treinar o Modelo

In [16]:
history = model.fit(ds_train,
                    epochs=10,
                    validation_data=ds_val)

Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 902s 2s/step - accuracy: 0.9702 - loss: 0.0809 - val_accuracy: 0.9856 - val_loss: 0.0418
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 911s 2s/step - accuracy: 0.9866 - loss: 0.0376 - val_accuracy: 0.9847 - val_loss: 0.0432
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 931s 2s/step - accuracy: 0.9917 - loss: 0.0246 - val_accuracy: 0.9850 - val_loss: 0.0426
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 920s 2s/step - accuracy: 0.9939 - loss: 0.0195 - val_accuracy: 0.9867 - val_loss: 0.0502
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 911s 2s/step - accuracy: 0.9946 - loss: 0.0167 - val_accuracy: 0.9841 - val_loss: 0.0632
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 874s 2s/step - accuracy: 0.9939 - loss: 0.0177 - val_accuracy: 0.9813 - val_loss: 0.0761
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 952s 2s/step - accuracy: 0.9959 - loss: 0.0133 - val_accuracy: 0.9834 - val_loss: 0.0680
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 929s 2s/step - accuracy: 0.9962 - loss: 0.0112 - val_accu

## Avaliar o Modelo

In [17]:
loss, accuracy = model.evaluate(ds_val)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

146/146 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.9833 - loss: 0.0685
Loss: 0.06506160646677017
Accuracy: 0.9836629629135132
